In [1]:
import cv2
import os
import json
import ast

import torch
from torch.utils.data import Dataset

import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models

import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary
from torchvision import models, transforms
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
!ls ../data

utils.py           vig_class_map.pkl  vig_test
vig_annotation.pkl vig_dl.lst         vig_train


In [3]:
!ls

VIG_EK.ipynb         test_labels.txt      vig_annotation.txt
class_annotation.txt train_labels.txt     vig_dl.lst


In [4]:
#!pip install -U scikit-learn
#!pip install opencv-python-headless torch torchvision
#!pip install pandas
#!pip install torchsummary

#### Get list of files for test & train:

In [5]:
# Directory paths for test and train videos
test_video_dir = '../data/vig_test/'
train_video_dir = '../data/vig_train/'

# Function to read and process a video
def read_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file '{video_path}'")
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Process each frame here
        # You can perform operations like displaying, saving frames, or analyzing them
        
        # Display the frame (for demonstration purposes)
        cv2.imshow('Video Frame', frame)
        
        # Press 'q' to exit the video playback
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Read and process test videos
test_videos = [os.path.join(test_video_dir, filename) for filename in os.listdir(test_video_dir) if filename.endswith('.mp4')]
print(test_videos)

# Read and process train videos
train_videos = [os.path.join(train_video_dir, filename) for filename in os.listdir(train_video_dir) if filename.endswith('.mp4')]
print(train_videos)

['../data/vig_test/OR926HIiUOs.mp4', '../data/vig_test/Snz2sQ5QxeE.mp4', '../data/vig_test/9FDzG0xNp5s.mp4', '../data/vig_test/L3a132_uApg.mp4', '../data/vig_test/samyE_r0sXs.mp4', '../data/vig_test/ZM6krn037N4.mp4', '../data/vig_test/Ju7Kg_H2iZQ.mp4', '../data/vig_test/0W1sNicXboU.mp4', '../data/vig_test/R0bAuQ3vfgA.mp4', '../data/vig_test/dMUoyb5VbeE.mp4', '../data/vig_test/v5IM8bUpeP8.mp4', '../data/vig_test/K3CenN7d30Y.mp4', '../data/vig_test/78R6KgsSPRk.mp4', '../data/vig_test/DFLDW-560HQ.mp4', '../data/vig_test/1hMZbDz2KH8.mp4', '../data/vig_test/402OOck5CAk.mp4', '../data/vig_test/CKJpM2qfH8M.mp4', '../data/vig_test/FtTzkBlkLJ4.mp4', '../data/vig_test/5etqUVLDcE4.mp4', '../data/vig_test/tFLGKmOa0dc.mp4', '../data/vig_test/DoRebW9poXU.mp4', '../data/vig_test/y3M7a7sPQAU.mp4', '../data/vig_test/UlxhUKmnfiQ.mp4', '../data/vig_test/QS31Gm9tIig.mp4', '../data/vig_test/KPyYUly5xCc.mp4', '../data/vig_test/NcIR-UiXJI0.mp4', '../data/vig_test/2ffgd7k0vI8.mp4', '../data/vig_test/MDa1ZVdDz

#### Save annotations in Dict:

In [6]:
# File paths
class_annotation_file = 'class_annotation.txt'
vig_annotation_file = 'vig_annotation.txt'

def read_annotation(file_path):

    # Read the contents of the text file
    with open(file_path, 'r') as file:
        file_contents = file.read()
    
    # Safely evaluate the contents as a Python dictionary
    try:
        data_dict = ast.literal_eval(file_contents)
    except (SyntaxError, ValueError):
        print("Error: The file does not contain a valid Python dictionary.")
        data_dict = {}

    return data_dict

class_annotation_dict = read_annotation(class_annotation_file)
vig_annotation_dict = read_annotation(vig_annotation_file)

print(class_annotation_dict)
print()
print(vig_annotation_dict)
print(vig_annotation_dict[3058])

{'Bleat': 2, 'Fireworks': 12, 'Splash, splatter': 13, 'Fire alarm': 9, 'Chicken, rooster': 3, 'Thunderstorm': 5, 'Gunshot, gunfire': 11, 'Rail transport': 7, 'Spray': 14, 'Race car, auto racing': 6, 'Church bell': 4, 'Helicopter': 8, 'Bark': 0, 'Hammer': 10, 'Cattle, bovinae': 1}

{327681: {'class_id': 1, 'start_time': 24.0, 'end_time': 34.0}, 425986: {'class_id': 11, 'start_time': 300.0, 'end_time': 310.0}, 13748: {'class_id': 11, 'start_time': 30.0, 'end_time': 40.0}, 955830: {'class_id': 13, 'start_time': 0.0, 'end_time': 10.0}, 65546: {'class_id': 0, 'start_time': 30.0, 'end_time': 40.0}, 11: {'class_id': 12, 'start_time': 370.0, 'end_time': 380.0}, 65549: {'class_id': 6, 'start_time': 20.0, 'end_time': 30.0}, 131086: {'class_id': 7, 'start_time': 40.0, 'end_time': 50.0}, 65555: {'class_id': 8, 'start_time': 90.0, 'end_time': 100.0}, 297879: {'class_id': 0, 'start_time': 60.0, 'end_time': 70.0}, 518830: {'class_id': 8, 'start_time': 30.0, 'end_time': 40.0}, 22: {'class_id': 10, 'st

#### Save vig_dl.lst into Dict:

In [7]:
vig_dl_dict = {}

# Open the file for reading
with open('vig_dl.lst', 'r') as file:
    # Read each line from the file
    for line in file:
        # Split the line by comma (,) to get the entries
        entries = line.strip().split(',')
        video_youtube_id = entries[0]
        video_int_id = int(entries[1])

        # Print the entries
        #print("video_youtube_id:", video_youtube_id)
        #print("video_int_id:", video_int_id)

        if video_youtube_id not in vig_dl_dict:
            vig_dl_dict[video_youtube_id] = video_int_id

print(vig_dl_dict)

{'kFZhc_d-2V4': 11, 'IcmAVLJQAXs': 22, 'njodYtK0Hqg': 36, 'LC4vDpnkDtM': 56, 'zNuT6b5vAnU': 66, '1XwOM3Hf_Bs': 68, 'OazWqV22fh4': 95, 'nvBPPOzcW-A': 99, 'qPsoKSYzuyo': 115, 'xvanhCQ9jho': 119, 'EOKUV6hgMhM': 132, 'QP_ZCssCySw': 147, 'BLEthZXy35s': 176, '9wMeb3Q-IiQ': 179, 'V2OjPsg7ydY': 190, 'VFjxWdPPY1Q': 195, 'M3tAFphkQow': 200, 'fVfkPyZl6o0': 220, 'kTyq_6AnoSw': 228, 'DIxTruxPbdk': 252, 'ITYq5xoDGK0': 297, 'd-do1XZ8f_E': 305, 'VWyrns4rBwc': 312, 'wuz0curb_hg': 315, 'XO1wG3ne_Cw': 354, 'qnUax_XrBC8': 359, 'Dks0c0Ii48o': 361, 'tz0avWZoqjg': 373, '7C_jcD_FOQI': 377, 'ldvGteIeoMQ': 399, 'POuhGn61ZR8': 428, 'lo3yvn1av2E': 437, 'DHkK8sw3dd0': 452, 'OfefVMM5L44': 459, 'ZnEEAfLC9Sg': 481, 'Amd-OLa3tMI': 500, 'ZKocFGL8LIE': 532, 'GO_1gBjIR1c': 550, 'tfMC-N8vX54': 580, 'aCX6vJhHO2c': 585, 'mTUsHkNjSWo': 594, '3EE5WKQMBek': 642, 'DpLi9YH5idY': 646, 'ItIo1RvO0PE': 659, 'v2i3nOEwupw': 668, '3hEgl8N2FBA': 685, 'qC5M7BAsKOA': 687, '0fsntL6RKww': 798, 'ijgwCwnKZUM': 812, 'fbJ0yNuwmpg': 829, 'vi6czb

#### Get train & test download assigned IDs:

#### train

In [8]:
# Directory path for train videos
train_video_dir = '../data/vig_train/'

# Get a list of all files in the train directory
file_list = os.listdir(train_video_dir)

# Remove the ".mp4" extension from each file name
file_list_without_extension = [os.path.splitext(file)[0] for file in file_list if file.endswith('.mp4')]

# Now, 'file_list_without_extension' contains the list of file names without the ".mp4" extension
print(file_list_without_extension[:4])

['GO_1gBjIR1c', 'Fzs-OIw-Atw', 'h6HkOxlVt5c', 'c9k4tTlQwmU']


In [9]:
# Define the output file
output_file = "train_labels.txt"

# Open the file in write mode
with open(output_file, "w") as file:
    
    for youtube_video_id in file_list_without_extension:
        video_id_value = vig_dl_dict.get(youtube_video_id, None)
        if video_id_value is not None:
            class_id = vig_annotation_dict.get(video_id_value, {}).get("class_id", 0)
            
            file.write(youtube_video_id + ".mp4," + str(class_id) + "\n")
            
            print("youtube_video_id:", youtube_video_id)
            print("video_id_value:", video_id_value)
            print("class_id:", class_id)
            print("-----------")


youtube_video_id: GO_1gBjIR1c
video_id_value: 550
class_id: 8
-----------
youtube_video_id: Fzs-OIw-Atw
video_id_value: 14375
class_id: 3
-----------
youtube_video_id: h6HkOxlVt5c
video_id_value: 3058
class_id: 3
-----------
youtube_video_id: c9k4tTlQwmU
video_id_value: 12928
class_id: 11
-----------
youtube_video_id: JNUF3mMSnSI
video_id_value: 5377
class_id: 4
-----------
youtube_video_id: t0txEC0Rhdg
video_id_value: 9519
class_id: 8
-----------
youtube_video_id: amKlD9GdFvM
video_id_value: 1054
class_id: 5
-----------
youtube_video_id: -oGJwGEsIiA
video_id_value: 12915
class_id: 13
-----------
youtube_video_id: TtP63IOhg1Y
video_id_value: 3167
class_id: 2
-----------
youtube_video_id: XO1wG3ne_Cw
video_id_value: 354
class_id: 0
-----------
youtube_video_id: IcmAVLJQAXs
video_id_value: 22
class_id: 10
-----------
youtube_video_id: Ey6ulciy5hs
video_id_value: 14255
class_id: 6
-----------
youtube_video_id: XyktNsq4SZU
video_id_value: 7010
class_id: 6
-----------
youtube_video_id: eXQY

In [10]:
# Directory path for train videos
test_video_dir = '../data/vig_test/'

# Get a list of all files in the train directory
file_list = os.listdir(test_video_dir)

# Remove the ".mp4" extension from each file name
file_list_without_extension = [os.path.splitext(file)[0] for file in file_list if file.endswith('.mp4')]

# Now, 'file_list_without_extension' contains the list of file names without the ".mp4" extension
print(file_list_without_extension[:4])

['OR926HIiUOs', 'Snz2sQ5QxeE', '9FDzG0xNp5s', 'L3a132_uApg']


#### test

In [11]:
# Define the output file
output_file = "test_labels.txt"

# Open the file in write mode
with open(output_file, "w") as file:
    
    for youtube_video_id in file_list_without_extension:
        video_id_value = vig_dl_dict.get(youtube_video_id, None)
        if video_id_value is not None:
            class_id = vig_annotation_dict.get(video_id_value, {}).get("class_id", 0)
            
            file.write(youtube_video_id + ".mp4," + str(class_id) + "\n")
            
            print("youtube_video_id:", youtube_video_id)
            print("video_id_value:", video_id_value)
            print("class_id:", class_id)
            print("-----------")

youtube_video_id: OR926HIiUOs
video_id_value: 20039
class_id: 7
-----------
youtube_video_id: Snz2sQ5QxeE
video_id_value: 18491
class_id: 7
-----------
youtube_video_id: 9FDzG0xNp5s
video_id_value: 18044
class_id: 7
-----------
youtube_video_id: L3a132_uApg
video_id_value: 18911
class_id: 7
-----------
youtube_video_id: samyE_r0sXs
video_id_value: 19578
class_id: 3
-----------
youtube_video_id: ZM6krn037N4
video_id_value: 20757
class_id: 5
-----------
youtube_video_id: Ju7Kg_H2iZQ
video_id_value: 18766
class_id: 6
-----------
youtube_video_id: 0W1sNicXboU
video_id_value: 19876
class_id: 13
-----------
youtube_video_id: R0bAuQ3vfgA
video_id_value: 18603
class_id: 12
-----------
youtube_video_id: dMUoyb5VbeE
video_id_value: 20903
class_id: 10
-----------
youtube_video_id: v5IM8bUpeP8
video_id_value: 17595
class_id: 0
-----------
youtube_video_id: K3CenN7d30Y
video_id_value: 20029
class_id: 3
-----------
youtube_video_id: 78R6KgsSPRk
video_id_value: 20481
class_id: 5
-----------
youtube_v

#### model training 

#### CNN (alextnet) and LSTM

In [12]:
# Define paths
labels_file = 'train_labels.txt'
video_directory = '../data/vig_train/'

# Load and preprocess data
labels = []
video_data = []

num_frames = 15

frames_list = []  # Store all frames as a list
labels_list = []  # Store corresponding labels

with open(labels_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        video_name, label = line.strip().split(',')
        labels_list.append(int(label))
        video_path = os.path.join(video_directory, video_name)

        if '.DS_Store' in video_path:
            continue

        cap = cv2.VideoCapture(video_path)

        frames = []

        for j in range(num_frames):
            frame_idx = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) * j / num_frames)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frame = frame / 255.0
            frames.append(frame)

        while len(frames) < num_frames:
            frames.append(frames[-1])

        frames_list.append(frames)

# Convert the lists to NumPy arrays
frames_array = np.array(frames_list)
labels_array = np.array(labels_list)


[h264 @ 0x296f821b0] mmco: unref short failure
[h264 @ 0x296f821b0] mmco: unref short failure
[h264 @ 0x296fcbc20] mmco: unref short failure
[h264 @ 0x296f6e1e0] mmco: unref short failure
[h264 @ 0x296fd99d0] mmco: unref short failure
[h264 @ 0x296fd99d0] mmco: unref short failure
[h264 @ 0x296f6cc90] mmco: unref short failure


In [13]:
# Define a function to apply transformations to frames (required for AlexNet)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Define the AlexNet model
alexnet_model = models.alexnet(pretrained=False, num_classes=15)

# Modify the classifier part of AlexNet to match your number of output classes
num_classes = 15  # Change this to your desired number of classes
alexnet_model.classifier[6] = nn.Linear(4096, num_classes)

print("Frames array shape:", frames_array.shape)
print("Labels array shape:", labels_array.shape)

#print(frames_array[:1])
#print(labels_array[:1])

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the output of the last time step
        return out

# Initialize the LSTM model
input_size = 4096  # The output feature size of AlexNet
hidden_size = 512  # You can adjust this as needed
num_layers = 2  # You can adjust this as needed
lstm_model = LSTMModel(input_size, hidden_size, num_layers, num_classes)

Frames array shape: (800, 15, 224, 224, 3)
Labels array shape: (800,)


In [14]:
class CombinedModel(nn.Module):
    def __init__(self, alexnet, lstm):
        super(CombinedModel, self).__init__()
        self.alexnet = alexnet
        self.lstm = lstm

    def forward(self, frames):
        # Extract features using AlexNet
        features = self.alexnet(frames)
        # Reshape features for LSTM input
        features = features.view(frames.size(0), -1)
        # Pass through LSTM
        output = self.lstm(features)
        return output

# Combine AlexNet and LSTM
combined_model = CombinedModel(alexnet_model, lstm_model)

In [15]:
best_accuracy = 0.0
best_model_state = None

# Reshape 'frames_array' to have the shape (num_samples, num_channels, height, width)
#frames_array = frames_array.reshape((3, 3, 224, 224))
#frames_array = frames_array.reshape((frames_array.shape[0], 3, 224, 224)) #may need to change

batch_size, num_frames, height, width, num_channels = frames_array.shape
frames_tensor = frames_array.reshape((batch_size * num_frames, 3, 224, 224)) 

print("frames_array.shape:", frames_array.shape)
print("frames_tensor.shape:", frames_tensor.shape)
print("labels_array.shape:", labels_array.shape)

# Convert data to PyTorch tensors
frames_tensor = torch.tensor(frames_tensor, dtype=torch.float32)
labels_array_new = [item for item in labels_array for _ in range(15)] 
labels_array_new = torch.tensor(labels_array_new, dtype=torch.long)

print("------")
print("labels_array_new.shape:", labels_array_new.shape)
print("frames_tensor.shape:", frames_tensor.shape)

# Create a TensorDataset
data = TensorDataset(frames_tensor, labels_array_new)

# Create a DataLoader
batch_size = 32  # You can adjust the batch size as needed
data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(combined_model.parameters(), lr=0.001, momentum=0.9)

# Lists to store predictions and true labels
class_names = ['Bark', 'Cattle_bovinae', 'Bleat', 'rooster', 'Churchbell', 'Thunderstorm', 'Racecar_autoracing', 'Railtransport', 'Helicopter', 'Firealarm', 'Hammer', 'Gunshot_gunfire', 'Fireworks', 'Splash,splatter', 'Spray']
all_predictions = []
all_labels = []


# Training loop
num_epochs = 100  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    for frames, labels in data_loader:
        optimizer.zero_grad()
        outputs = alexnet_model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Convert model outputs to class predictions
        _, predicted = torch.max(outputs, 1)
        
        # Append the predictions and labels to the lists
        all_predictions.extend(predicted.tolist())
        all_labels.extend(labels.tolist())

    # Calculate the confusion matrix
    display(pd.DataFrame(confusion_matrix(all_labels, all_predictions), columns=class_names))
    
    # Calculate the accuracy rate
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f"Accuracy Rate: {accuracy * 100:.2f}%")
    # Check if the current accuracy is better than the best accuracy
    if accuracy > best_accuracy:
        
        best_accuracy = accuracy
        best_model_state = alexnet_model.state_dict()  # Save the model state


frames_array.shape: (800, 15, 224, 224, 3)
frames_tensor.shape: (12000, 3, 224, 224)
labels_array.shape: (800,)
------
labels_array_new.shape: torch.Size([12000])
frames_tensor.shape: torch.Size([12000, 3, 224, 224])


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,0,0,0,0,0,0,667,0,0,20,3,0,0,0
1,0,0,0,0,0,0,0,630,0,0,27,3,0,0,0
2,0,0,0,0,0,0,0,580,0,0,18,2,0,0,0
3,0,0,0,0,0,0,0,800,0,0,39,1,0,0,0
4,0,0,0,0,0,0,0,588,0,0,24,3,0,0,0
5,0,0,0,0,0,0,0,580,0,0,19,1,0,0,0
6,0,0,0,0,0,0,0,823,0,0,32,0,0,0,0
7,0,0,0,0,0,0,0,1792,0,0,78,5,0,0,0
8,0,0,0,0,0,0,0,630,0,0,27,3,0,0,0
9,0,0,0,0,0,0,0,587,0,0,24,4,0,0,0


Accuracy Rate: 15.26%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,0,0,0,0,0,0,1354,0,0,20,6,0,0,0
1,0,0,0,0,0,0,0,1286,0,0,27,7,0,0,0
2,0,0,0,0,0,0,0,1176,0,0,18,6,0,0,0
3,0,0,0,0,0,0,0,1633,0,0,39,8,0,0,0
4,0,0,0,0,0,0,0,1200,0,0,24,6,0,0,0
5,0,0,0,0,0,0,0,1180,0,0,19,1,0,0,0
6,0,0,0,0,0,0,0,1676,0,0,32,2,0,0,0
7,0,0,0,0,0,0,0,3653,0,0,78,19,0,0,0
8,0,0,0,0,0,0,0,1286,0,0,27,7,0,0,0
9,0,0,0,0,0,0,0,1199,0,0,24,7,0,0,0


Accuracy Rate: 15.41%


...

Accuracy Rate: 77.99%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,51182,400,402,496,127,322,791,4532,55,396,181,8200,730,211,285
1,606,49476,85,276,42,95,590,6916,183,24,392,5525,302,441,387
2,853,166,42983,681,78,223,341,5002,83,249,369,7064,490,508,310
3,104,22,167,64898,190,489,316,5626,244,701,324,8722,433,97,827
4,70,86,20,744,46080,548,94,4409,71,238,309,6916,734,72,494
5,79,32,34,722,96,43797,382,3938,254,309,34,7772,1696,14,241
6,764,364,284,1103,65,496,63220,8895,500,303,193,7686,139,180,453
7,379,634,510,807,348,2703,837,164757,418,99,1026,10484,1600,493,530
8,73,165,109,1048,154,385,886,7324,47670,269,354,5649,263,241,750
9,278,67,123,1635,441,1434,403,3436,222,42592,114,8767,887,161,325


Accuracy Rate: 78.20%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,51871,400,402,496,127,323,791,4532,55,396,181,8200,730,211,285
1,606,50136,85,276,42,95,590,6916,183,24,392,5525,302,441,387
2,853,166,43582,681,78,224,341,5002,83,249,369,7064,490,508,310
3,104,22,167,65738,190,489,316,5626,244,701,324,8722,433,97,827
4,70,86,20,744,46691,550,94,4411,71,238,309,6916,734,72,494
5,79,32,34,722,96,44389,382,3945,254,309,34,7773,1696,14,241
6,764,364,284,1103,65,497,64074,8895,500,303,193,7686,139,180,453
7,379,634,510,807,348,2731,837,166604,418,99,1026,10484,1600,493,530
8,73,165,109,1048,154,385,886,7324,48330,269,354,5649,263,241,750
9,278,67,123,1635,441,1444,403,3439,222,43194,114,8767,887,161,325


Accuracy Rate: 78.42%


In [16]:
# Save the model with the best accuracy
if best_model_state is not None:
    best_model = models.alexnet(pretrained=False, num_classes=num_classes)  # Create a new model instance
    best_model.load_state_dict(best_model_state)  # Load the best model state
    torch.save(best_model, 'cnn_lstm_best_model.pth')

In [17]:
# Load the trained model
loaded_model = torch.load('cnn_lstm_best_model.pth')
loaded_model.eval()  # Set the model in evaluation mode

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [18]:
# Define paths
test_labels_file = 'test_labels.txt'
test_video_directory = '../data/vig_test/'


num_frames = 15

test_frames_list = []  # Store all frames as a list
test_labels_list = []  # Store corresponding labels

with open(test_labels_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        video_name, label = line.strip().split(',')
        test_labels_list.append(int(label))
        video_path = os.path.join(test_video_directory, video_name)

        if '.DS_Store' in video_path:
            continue

        cap = cv2.VideoCapture(video_path)

        frames = []

        for j in range(num_frames):
            frame_idx = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) * j / num_frames)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frame = frame / 255.0
            frames.append(frame)

        while len(frames) < num_frames:
            frames.append(frames[-1])

        test_frames_list.append(frames)

# Convert the lists to NumPy arrays
test_frames_array = np.array(test_frames_list)
test_labels_array = np.array(test_labels_list)

print(test_frames_array.shape)
print(test_labels_array.shape)


batch_size, num_frames, height, width, num_channels = test_frames_array.shape
test_frames_tensor = test_frames_array.reshape((batch_size * num_frames, 3, 224, 224)) 

print("test_frames_array.shape:", test_frames_array.shape)
print("test_frames_tensor.shape:", test_frames_tensor.shape)
print("test_labels_array.shape:", test_labels_array.shape)


# Convert data to PyTorch tensors
test_frames_tensor = torch.tensor(test_frames_tensor, dtype=torch.float32)
test_labels_array_new = [item for item in test_labels_array for _ in range(15)] 
test_labels_array_new = torch.tensor(test_labels_array_new, dtype=torch.long)

print("------")
print("test_labels_array_new.shape:", test_labels_array_new.shape)
print("test_frames_tensor.shape:", test_frames_tensor.shape)


# Create a DataLoader for the test data
test_data = TensorDataset(test_frames_tensor, test_labels_array_new)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Lists to store predictions and true labels for test data
test_all_predictions = []
test_all_labels = []

for frames, labels in test_data_loader:
    # Perform inference using the loaded model
    test_outputs = loaded_model(frames)
    
    # Convert model outputs to class predictions
    _, test_predicted = torch.max(test_outputs, 1)
    
    # Append the predictions and labels to the test lists
    test_all_predictions.extend(test_predicted.tolist())
    test_all_labels.extend(labels.tolist())

# Calculate the confusion matrix for test data
test_confusion_matrix = confusion_matrix(test_all_labels, test_all_predictions)
display(pd.DataFrame(test_confusion_matrix, columns=class_names))

# Calculate the accuracy rate for test data
test_accuracy = accuracy_score(test_all_labels, test_all_predictions)
print(f"Test Accuracy Rate: {test_accuracy * 100:.2f}%")


[h264 @ 0x11016b6e0] mmco: unref short failure
[h264 @ 0x11016b6e0] mmco: unref short failure


(163, 15, 224, 224, 3)
(163,)
test_frames_array.shape: (163, 15, 224, 224, 3)
test_frames_tensor.shape: (2445, 3, 224, 224)
test_labels_array.shape: (163,)
------
test_labels_array_new.shape: torch.Size([2445])
test_frames_tensor.shape: torch.Size([2445, 3, 224, 224])


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,0,13,10,0,0,2,19,18,0,13,5,0,10,0
1,0,1,0,16,0,6,1,42,0,5,0,9,4,0,6
2,15,10,1,0,0,8,7,27,3,12,2,20,0,2,13
3,1,0,1,11,0,15,26,15,0,22,1,3,0,3,7
4,5,1,5,0,0,30,40,26,3,3,23,15,15,10,4
5,6,15,0,15,0,50,17,6,0,7,0,15,34,0,0
6,3,0,9,2,0,0,52,10,10,22,1,2,0,5,4
7,13,25,23,19,25,21,59,231,8,32,17,36,7,23,16
8,0,0,7,4,1,3,32,33,15,5,1,11,15,22,1
9,21,3,19,5,13,0,6,3,1,8,2,15,4,17,3


Test Accuracy Rate: 19.10%


#### CNN (alextnet)

In [2]:
# Define paths
labels_file = 'train_labels.txt'
video_directory = '../data/vig_train/'

# Load and preprocess data
labels = []
video_data = []

num_frames = 15

frames_list = []  # Store all frames as a list
labels_list = []  # Store corresponding labels

with open(labels_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        video_name, label = line.strip().split(',')
        labels_list.append(int(label))
        video_path = os.path.join(video_directory, video_name)

        if '.DS_Store' in video_path:
            continue

        cap = cv2.VideoCapture(video_path)

        frames = []

        for j in range(num_frames):
            frame_idx = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) * j / num_frames)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frame = frame / 255.0
            frames.append(frame)

        while len(frames) < num_frames:
            frames.append(frames[-1])

        frames_list.append(frames)

# Convert the lists to NumPy arrays
frames_array = np.array(frames_list)
labels_array = np.array(labels_list)

# Define a function to apply transformations to frames (required for AlexNet)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Define the AlexNet model
alexnet_model = models.alexnet(pretrained=False, num_classes=15)

# Modify the classifier part of AlexNet to match your number of output classes
num_classes = 15  # Change this to your desired number of classes
alexnet_model.classifier[6] = nn.Linear(4096, num_classes)

print("Frames array shape:", frames_array.shape)
print("Labels array shape:", labels_array.shape)

#print(frames_array[:1])
#print(labels_array[:1])

[h264 @ 0x28f635bc0] mmco: unref short failure
[h264 @ 0x28f635bc0] mmco: unref short failure
[h264 @ 0x28f616a00] mmco: unref short failure
[h264 @ 0x1694c6e40] mmco: unref short failure
[h264 @ 0x28dd9dc30] mmco: unref short failure
[h264 @ 0x28dd9dc30] mmco: unref short failure
[h264 @ 0x28f62b250] mmco: unref short failure


Frames array shape: (800, 15, 224, 224, 3)
Labels array shape: (800,)


In [3]:
best_accuracy = 0.0
best_model_state = None

# Reshape 'frames_array' to have the shape (num_samples, num_channels, height, width)
#frames_array = frames_array.reshape((3, 3, 224, 224))
#frames_array = frames_array.reshape((frames_array.shape[0], 3, 224, 224)) #may need to change

batch_size, num_frames, height, width, num_channels = frames_array.shape
frames_tensor = frames_array.reshape((batch_size * num_frames, 3, 224, 224)) 

print("frames_array.shape:", frames_array.shape)
print("frames_tensor.shape:", frames_tensor.shape)
print("labels_array.shape:", labels_array.shape)

# Convert data to PyTorch tensors
frames_tensor = torch.tensor(frames_tensor, dtype=torch.float32)
labels_array_new = [item for item in labels_array for _ in range(15)] 
labels_array_new = torch.tensor(labels_array_new, dtype=torch.long)

print("------")
print("labels_array_new.shape:", labels_array_new.shape)
print("frames_tensor.shape:", frames_tensor.shape)

# Create a TensorDataset
data = TensorDataset(frames_tensor, labels_array_new)

# Create a DataLoader
batch_size = 32  # You can adjust the batch size as needed
data_loader = DataLoader(data, batch_size=batch_size, shuffle=True)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet_model.parameters(), lr=0.001, momentum=0.9)

# Lists to store predictions and true labels
class_names = ['Bark', 'Cattle_bovinae', 'Bleat', 'rooster', 'Churchbell', 'Thunderstorm', 'Racecar_autoracing', 'Railtransport', 'Helicopter', 'Firealarm', 'Hammer', 'Gunshot_gunfire', 'Fireworks', 'Splash,splatter', 'Spray']
all_predictions = []
all_labels = []

# Training loop
num_epochs = 50  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    for frames, labels in data_loader:
        optimizer.zero_grad()
        outputs = alexnet_model(frames)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Convert model outputs to class predictions
        _, predicted = torch.max(outputs, 1)
        
        # Append the predictions and labels to the lists
        all_predictions.extend(predicted.tolist())
        all_labels.extend(labels.tolist())

    # Calculate the confusion matrix
    display(pd.DataFrame(confusion_matrix(all_labels, all_predictions), columns=class_names))
    
    # Calculate the accuracy rate
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f"Accuracy Rate: {accuracy * 100:.2f}%")
    # Check if the current accuracy is better than the best accuracy
    if accuracy > best_accuracy:
        
        best_accuracy = accuracy
        best_model_state = alexnet_model.state_dict()  # Save the model state


frames_array.shape: (800, 15, 224, 224, 3)
frames_tensor.shape: (12000, 3, 224, 224)
labels_array.shape: (800,)
------
labels_array_new.shape: torch.Size([12000])
frames_tensor.shape: torch.Size([12000, 3, 224, 224])


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,0,0,0,0,0,5,530,0,0,0,144,11,0,0
1,0,0,0,1,0,0,2,504,0,0,0,146,7,0,0
2,0,0,0,0,0,0,4,461,0,0,0,128,7,0,0
3,0,0,0,1,0,0,9,639,0,0,0,182,9,0,0
4,0,0,0,1,0,0,2,462,0,0,0,145,5,0,0
5,0,0,0,1,0,0,5,457,0,0,0,131,6,0,0
6,0,0,0,0,0,0,3,650,0,0,0,196,6,0,0
7,0,0,0,3,0,0,14,1426,0,0,0,416,16,0,0
8,0,0,0,0,0,0,2,493,0,0,0,164,1,0,0
9,0,0,0,1,0,0,2,475,0,0,0,127,10,0,0


Accuracy Rate: 15.06%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,0,0,0,0,0,5,1220,0,0,0,144,11,0,0
1,0,0,0,1,0,0,2,1164,0,0,0,146,7,0,0
2,0,0,0,0,0,0,4,1061,0,0,0,128,7,0,0
3,0,0,0,1,0,0,9,1479,0,0,0,182,9,0,0
4,0,0,0,1,0,0,2,1077,0,0,0,145,5,0,0
5,0,0,0,1,0,0,5,1056,0,0,0,132,6,0,0
6,0,0,0,0,0,0,3,1505,0,0,0,196,6,0,0
7,0,0,0,3,0,0,14,3301,0,0,0,416,16,0,0
8,0,0,0,0,0,0,2,1153,0,0,0,164,1,0,0
9,0,0,0,1,0,0,2,1090,0,0,0,127,10,0,0


Accuracy Rate: 15.34%


...

Accuracy Rate: 54.86%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,17267,719,413,227,99,111,750,5121,51,91,247,7846,623,178,67
1,1305,15115,257,150,23,84,375,8064,197,18,595,5246,452,241,218
2,1174,451,12564,325,78,46,496,5836,38,95,340,6901,559,398,99
3,46,2,27,22773,110,479,378,6252,268,634,41,8945,388,209,608
4,99,21,8,451,14792,270,131,4496,55,256,149,8046,998,59,304
5,122,3,10,1478,78,12923,150,3587,269,214,8,7857,2434,16,251
6,842,151,322,981,98,272,20091,9973,363,271,88,7599,245,312,287
7,402,900,480,837,126,971,966,71390,722,146,973,10984,2193,267,518
8,66,105,53,879,48,213,438,8117,14985,51,189,5442,499,398,857
9,204,35,132,1911,444,728,454,3844,163,11289,105,9311,914,254,347


Accuracy Rate: 55.71%


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,17929,722,416,229,101,112,752,5123,51,94,247,7846,631,180,67
1,1308,15758,257,150,23,84,376,8069,198,18,597,5247,453,244,218
2,1176,453,13140,328,78,47,498,5838,38,96,341,6904,561,403,99
3,47,2,27,23591,110,481,384,6253,271,639,41,8945,388,211,610
4,100,22,8,451,15385,273,132,4496,55,257,150,8052,1004,59,306
5,122,3,10,1478,78,13498,152,3592,269,218,8,7862,2443,16,251
6,848,153,323,983,98,277,20913,9977,366,276,88,7600,248,313,287
7,403,903,485,837,126,996,966,73195,727,148,976,10990,2209,269,520
8,66,106,53,883,49,214,440,8119,15631,51,189,5442,499,400,858
9,206,35,135,1919,446,742,457,3846,166,11855,105,9317,918,255,348


Accuracy Rate: 56.52%


#### save model

In [4]:
# Save the model with the best accuracy
if best_model_state is not None:
    best_model = models.alexnet(pretrained=False, num_classes=num_classes)  # Create a new model instance
    best_model.load_state_dict(best_model_state)  # Load the best model state
    torch.save(best_model, 'cnn_best_model.pth')

In [5]:
# Load the trained model
loaded_model = torch.load('cnn_best_model.pth')
loaded_model.eval()  # Set the model in evaluation mode

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# Define paths
test_labels_file = 'test_labels.txt'
test_video_directory = '../data/vig_test/'


num_frames = 15

test_frames_list = []  # Store all frames as a list
test_labels_list = []  # Store corresponding labels

with open(test_labels_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        video_name, label = line.strip().split(',')
        test_labels_list.append(int(label))
        video_path = os.path.join(test_video_directory, video_name)

        if '.DS_Store' in video_path:
            continue

        cap = cv2.VideoCapture(video_path)

        frames = []

        for j in range(num_frames):
            frame_idx = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) * j / num_frames)
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            frame = frame / 255.0
            frames.append(frame)

        while len(frames) < num_frames:
            frames.append(frames[-1])

        test_frames_list.append(frames)

# Convert the lists to NumPy arrays
test_frames_array = np.array(test_frames_list)
test_labels_array = np.array(test_labels_list)

print(test_frames_array.shape)
print(test_labels_array.shape)


batch_size, num_frames, height, width, num_channels = test_frames_array.shape
test_frames_tensor = test_frames_array.reshape((batch_size * num_frames, 3, 224, 224)) 

print("test_frames_array.shape:", test_frames_array.shape)
print("test_frames_tensor.shape:", test_frames_tensor.shape)
print("test_labels_array.shape:", test_labels_array.shape)


# Convert data to PyTorch tensors
test_frames_tensor = torch.tensor(test_frames_tensor, dtype=torch.float32)
test_labels_array_new = [item for item in test_labels_array for _ in range(15)] 
test_labels_array_new = torch.tensor(test_labels_array_new, dtype=torch.long)

print("------")
print("test_labels_array_new.shape:", test_labels_array_new.shape)
print("test_frames_tensor.shape:", test_frames_tensor.shape)


# Create a DataLoader for the test data
test_data = TensorDataset(test_frames_tensor, test_labels_array_new)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Lists to store predictions and true labels for test data
test_all_predictions = []
test_all_labels = []

for frames, labels in test_data_loader:
    # Perform inference using the loaded model
    test_outputs = loaded_model(frames)
    
    # Convert model outputs to class predictions
    _, test_predicted = torch.max(test_outputs, 1)
    
    # Append the predictions and labels to the test lists
    test_all_predictions.extend(test_predicted.tolist())
    test_all_labels.extend(labels.tolist())

# Calculate the confusion matrix for test data
test_confusion_matrix = confusion_matrix(test_all_labels, test_all_predictions)
display(pd.DataFrame(test_confusion_matrix, columns=class_names))

# Calculate the accuracy rate for test data
test_accuracy = accuracy_score(test_all_labels, test_all_predictions)
print(f"Test Accuracy Rate: {test_accuracy * 100:.2f}%")


[h264 @ 0x28f321c30] mmco: unref short failure
[h264 @ 0x28f321c30] mmco: unref short failure


(163, 15, 224, 224, 3)
(163,)
test_frames_array.shape: (163, 15, 224, 224, 3)
test_frames_tensor.shape: (2445, 3, 224, 224)
test_labels_array.shape: (163,)
------
test_labels_array_new.shape: torch.Size([2445])
test_frames_tensor.shape: torch.Size([2445, 3, 224, 224])


,Bark,Cattle_bovinae,Bleat,rooster,Churchbell,Thunderstorm,Racecar_autoracing,Railtransport,Helicopter,Firealarm,Hammer,Gunshot_gunfire,Fireworks,"Splash,splatter",Spray
0,0,11,0,10,3,0,7,15,2,19,0,14,0,3,6
1,0,1,0,18,1,0,2,54,0,0,0,12,0,0,2
2,0,0,1,16,3,0,9,37,7,15,1,7,0,2,22
3,0,0,0,6,3,2,1,31,16,34,0,0,0,9,3
4,0,23,0,0,11,15,11,81,0,4,1,15,0,14,5
5,0,15,9,30,0,40,15,0,0,30,0,19,7,0,0
6,14,6,6,3,8,0,31,20,4,13,0,3,0,5,7
7,7,27,13,27,12,22,68,204,50,44,16,35,1,8,21
8,5,5,10,2,6,6,16,25,26,2,4,15,15,13,0
9,5,3,9,1,6,3,7,7,0,19,2,40,0,14,4


Test Accuracy Rate: 17.01%


Reading:

https://towardsdatascience.com/the-w3h-of-alexnet-vggnet-resnet-and-inception-7baaaecccc96

https://blog.paperspace.com/alexnet-pytorch/

https://theaisummer.com/cnn-architectures/

https://pytorch.org/hub/pytorch_vision_alexnet/